In [215]:
# import all the required torch functions
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [216]:
# This is out base model for MNIST classification
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()

        # Each Convolution extracts feature, what feature depends on the Kernel values. Kernel is generally Odd Size.
        # prototype => torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, etc

        # Max pooling is a pooling operation that selects the maximum element from the region of the feature map covered by the filter. 
        # Thus, the output after max-pooling layer would be a feature map containing the most prominent features of the previous feature map.
        # The movement we do max pooling the RF(receptive field) doubles.

        # Input_Size: 28x28(as per the caller in this example)
        
        # input: 28x28x1 => output: 22x22x32
        self.conv1 = nn.Sequential(
            
            # the digits are focused around the center, so no need to maintain pixel boundary with paadding = 1
            # Input_Size: 28x28(depends on image input) => Input_Channel: 1, Kernel: 3x3x1, Padding: 0, Output_Channels/No_Of_Kernels: 16 (3x3x1(16))=> Output: 26x26x16
            nn.Conv2d(1, 16, 3, bias=False), # 28x28x1 -> 26x26x16 ? RF: 3x3
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            nn.Conv2d(16, 16, 3, bias=False), # 26x26x16 -> 24x24x16 ? RF: 5x5
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            nn.Conv2d(16, 32, 3, bias=False), # 24x24x16 -> 22x22x32 ? RF: 7x7
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),
        )

        # translation layer: merge features using 1x1 convolution
        # input: 22x22x32 => output: 11x11x16
        self.trans1 = nn.Sequential(
        
            nn.Conv2d(32, 16, 1, bias=False), # 22x22x32 -> 22x22x16 ? RF: 7x7 (remains same with 1x1 conv2d)
            nn.ReLU(),
            #nn.BatchNorm2d(16),

            nn.MaxPool2d(2, 2), # 22x22x16 -> 11x11x16 ? RF: 14x14 (doubled)
        )
        
        # input: 11x11x16 => output: 7x7x16
        self.conv2 = nn.Sequential(

            nn.Conv2d(16, 16, 3, bias=False), # 11x11x16 -> 9x9x16 ? RF: 16x16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            nn.Conv2d(16, 16, 3, bias=False), # 9x9x16 -> 7x7x16 ? RF: 18x18
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
        )

       # input: 7x7x16 => output: 5x5x16
        self.conv3 = nn.Sequential(

            nn.Conv2d(16, 16, 3, padding=1, bias=False), # 7x7x16 -> 7x7x16 ? RF: 20x20
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            nn.Conv2d(16, 16, 3, bias=False), # 7x7x16 -> 5x5x16 ? RF: 22x22 
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
        )

        # GAP Layer
        # input: 5x5x16 => output: 1x1x10
        self.gap_layer = nn.Sequential(
            # 1x1 convoluation to mix 16 channels
            nn.Conv2d(16, 10, 1, bias=False), # 5x5x16 -> 5x5x10 ? RF: 22x22
            nn.AvgPool2d(5) # 5x5x10 -> 1x1x10 
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.trans1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.gap_layer(x)

        x = x.view(-1, 10) # Currently the output is 1x1x10. Flatten it to 1x10 before feeding it to Log_softmax (-1 identified the other dimension, knowing one dimension Ex.10)
        return F.log_softmax(x)

In [217]:
# Torch-summary provides information complementary to what is provided by print(your_model) in PyTorch, 
# similar to Tensorflow's model. summary() API to view the visualization of the model, which is helpful while debugging your network.
!pip install torchsummary
from torchsummary import summary

# Once we have GPU ready to access, the next step is getting PyTorch to use for storing data (tensors) and computing on data (performing operations on tensors).
# test if PyTorch has access to a GPU
use_cuda = torch.cuda.is_available()

# if GPU is availble use it, if not use CPU. allocate device and assign n/w to it
device = torch.device("cuda" if use_cuda else "cpu")

# configure the model to be loaded on the device decided above.
model = Net().to(device)

# input image size: 28x28, Input_channels: 1
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,608
             ReLU-10           [-1, 32, 22, 22]               0
      BatchNorm2d-11           [-1, 32, 22, 22]              64
        Dropout2d-12           [-1, 32, 22, 22]               0
    

<ipython-input-216-30a5a40f7c06>:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [218]:

#  When requested for random number in pytorch (with torch.rand() etc) these random numbers are generated from a specific algorithm. 
# A nice property of this algorithm is that we can fix it's starting point and it will always generate the same random numbers afterwards
# seed is used to fix the starting point. This helps to start always at the same point every time we restart. Helps a lot to evaluate results taken at different times.
torch.manual_seed(1)

# This is the batch size required for training/evaluation at a time.
batch_size = 128

# to configure the workers required to collect the data
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Load required datasets: Kwargs help to accelerate the collection.
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [219]:
# Tqdm is a library in Python which is used for creating Progress Meters or Progress Bars.
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    
    model.train() # set the model with training mode (this is the default mode)

    # load the training data and show the progress bar. Useful when data is big.
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)   # connect data and target(labels) to same device
        optimizer.zero_grad()                     # reset all optimizer's gradients before new pass. To be recalculated for the specific training step
        output = model(data)                      # The model goes through all of the training data in this batch once, performing its forward() function calculations. 
        loss = F.nll_loss(output, target)         # Calculate the loss, output w.r.t target(ground truth), nll - Negative Log-Likelihood Loss function - applied only on models with the softmax function as an output activation layer.
       
        loss.backward()                           # Computes the gradient of the loss with respect for every model parameter to be updated (each parameter with requires_grad=True). This is known as backpropagation, hence "backwards".
        optimizer.step()                          # Update the parameters with requires_grad=True with respect to the loss gradients in order to improve them
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')  # update current status 


def test(model, device, test_loader):
    model.eval()    # set the model with eval mode 
    test_loss = 0
    correct = 0 
    with torch.no_grad():    # anything done within this context does not update gradients, making the computation efficient     
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)     # connect data and target to same device
            output = model(data)                                  # The model goes through all of the test data in this batch once, performing its forward() function calculations.
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # cacluate the loss w.r.t target, add up the  batch-loss as part of the total loss (lower the better)
            pred = output.argmax(dim=1, keepdim=True)             # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item() # check if the predicted and target is same within the batch, if so how many (sum())

    test_loss /= len(test_loader.dataset)   # Average test loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [220]:

# move the model to the required device, GPU if available, if not on CPU.
model = Net().to(device)

# create an optimizer (SGD) with required parameters, 
# model.parameters() - these are the model parameters to optimize, 
# learning-rate lr=0.01 and momentum-0.9
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Configure epochs to train for
EPOCH = 20

# run single epoch as of now.
for epoch in range(1, EPOCH+1):
    print(f'Epoch: {epoch}')

    # train the model loaded on the device with the required optimizer
    train(model, device, train_loader, optimizer, epoch)

    # we need to evaluate on test data
    test(model, device, test_loader)

Epoch: 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-216-30a5a40f7c06>:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.27887117862701416 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.35it/s]



Test set: Average loss: 0.0716, Accuracy: 9788/10000 (97.88%)

Epoch: 2


loss=0.0994519516825676 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.58it/s]



Test set: Average loss: 0.0420, Accuracy: 9881/10000 (98.81%)

Epoch: 3


...


Test set: Average loss: 0.0189, Accuracy: 9940/10000 (99.40%)

Epoch: 19


loss=0.05251206085085869 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.21it/s]



Test set: Average loss: 0.0167, Accuracy: 9948/10000 (99.48%)

Epoch: 20


loss=0.03706631436944008 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.65it/s]



Test set: Average loss: 0.0164, Accuracy: 9947/10000 (99.47%)



# Original Model
Model Results
- Total params: 6,379,786
- Trainable params: 6,379,786
- Non-trainable params: 0
- Forward/backward pass size (MB): 1.51
- Params size (MB): 24.34
- Estimated Total Size (MB): 25.85
- Accuracy: 28%

---

# Overall Improvements Done

## Improvements Done:
- In Class Net Model
  - Started with lower channel size, removed padding to reduce image 
  - same time consistently increased Receptive field (RF), not more than image size
  - Removed Relu just before the end layer so that we consider both +ve & -ve values
  - Included Batch Normalization
  - Included Dropout of 10%
  - Include Gap layer at the end instead of FC
  - Used Maxpool far away from the last layer
  - Utilized 1x1 convulations to merge results of conv2d to extract better features

- In Training
  - Increased Number of Epocs

## Model Results
- Total params: 17,200
- Trainable params: 17,200
- Non-trainable params: 0
- Forward/backward pass size (MB): 1.32
- Params size (MB): 0.07
- Estimated Total Size (MB): 1.39
- Accuracy: 99.47%